In [426]:
using Pkg
Pkg.activate("/home/fedflorio/master_thesis/")

  Activating project at `~/master_thesis`


In [427]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, PyPlot, DelimitedFiles
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2;

In [428]:
A = readdlm("/home/fedflorio/master_thesis/MatrixProductBP.jl/notebooks/sis_inference_data/graphs/ER.txt", Bool)
g = IndexedGraph(A)

nsnaps = 100
separation = 32
T = nsnaps * separation
N = nv(g)
seed = 8

λ_unif = 0.2
ρ_unif = 0.2
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        # λ[i] = rand()
        λ[i] = λ_unif
    end
end
λ = sparse(λ)
# ρ = rand(N)
ρ = fill(ρ_unif,N)
γ = 0.1
;

In [429]:
sis = SIS_heterogeneous(λ, ρ, T; γ)
bp_obs = mpbp(sis);

In [430]:
obs_times = collect(range(separation, step=separation, length=nsnaps))
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp_obs, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [431]:
println(any(!=(1), X[:,end]))
X

true


30×3201 Matrix{Int64}:
 1  1  1  2  2  1  1  1  1  1  1  1  2  …  2  1  2  2  1  1  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  2  2  2  2  1     1  1  1  2  2  2  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  2  2  2  2  2  2  2  2  2     2  2  2  2  2  2  1  2  2  2  2  2
 1  1  2  2  2  2  2  2  1  2  2  2  2     2  2  2  2  2  2  2  2  2  2  1  1
 1  1  1  1  1  1  1  1  1  1  1  2  2  …  2  2  2  2  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  2     1  1  1  1  1  2  2  2  2  2  2  2
 1  1  1  2  2  1  1  1  1  1  1  1  1     2  2  1  1  1  1  1  1  1  2  2  1
 1  1  1  1  1  1  1  2  2  2  2  2  1     1  1  1  1  2  2  2  2  1  1  1  1
 1  1  1  2  2  2  1  2  2  2  1  2  2  …  2  2  2  2  2  1  2  2  2  2  2  1
 1  1  1  1  2  1  1  1  1  2  2  1  1     1  1  1  1  2  2  2  1  2  2  1  2
 2  2  2  2  2  2  2  2  2  2  2  2  2   